In [1]:
USER = 'Nunzio'
LOSS = 'mean_euclidean_error'

In [2]:
import sys
import os
if (colab := 'google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = '/content/drive/Shareddrives/Project_ML_23/' + USER + '/machine-learning-project'
    sys.path.insert(0,BASE_PATH)
    N_JOBS = 1
    sys.path.insert(0,BASE_PATH + '/src/utils')
    !pip install optuna
    !pip install scikit-learn
    !pip install scikeras
    TRAIN_DATA = os.path.join(BASE_PATH, 'datasets', 'cup2023','ML-CUP23-TR.csv')
    IMAGES_FOLDER = os.path.join(BASE_PATH, 'images', 'cup2023', 'neural_network')
    MODEL_FOLDER = os.path.join(BASE_PATH, 'trained_models', 'cup2023')
else :
    N_JOBS = -1
    TRAIN_DATA = os.path.join('..', '..', 'datasets', 'cup2023', 'ML-CUP23-TR.csv')
    IMAGES_FOLDER = os.path.join('..', '..', 'images', 'cup2023', 'neural_network')
    MODEL_FOLDER = os.path.join('..', '..', 'trained_models', 'cup2023')

In [3]:
if (colab := 'google.colab' in sys.modules):
    sys.path.append(BASE_PATH + '/src/utils')
else:
    sys.path.append('../utils')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(style='darkgrid')
from keras import backend as K


from utils import save_plot, mean_euclidean_error, root_mean_squared_error, multidim_r2, set_random_state
from NN import NeuralNetwork
from joblib import dump

set_random_state(42)

/Users/nunzio/miniforge3/envs/ML23/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 14))

df_train = pd.read_csv(TRAIN_DATA, header=None, comment='#', usecols=columns_to_read, delimiter=',')
df_train = df_train.astype('float64') # casting
df_train.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251


In [5]:
features = ['feature_' + str(i) for i in range(1, 11)]
targets = ['target_x', 'target_y', 'target_z']

# Rename columns
new_column_names = features + targets
df_train.columns = new_column_names

df_train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,target_x,target_y,target_z
0,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251


In [6]:
from sklearn.model_selection import train_test_split

X = df_train[features].to_numpy()
y = df_train[targets].to_numpy()

# TODO split potenzialmente variabile?
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create model

In [7]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('neuralnetwork', NeuralNetwork(loss=LOSS, verbose=0))
])

In [8]:
param_grid = {
    'neuralnetwork__architecture': [
        (128, 64),
        (128, 128),
        (256, 128)
    ],
    'neuralnetwork__activation': ['relu'],
    'neuralnetwork__learning_rate': [1e-3, 1e-2],
    'neuralnetwork__momentum': [0.5,  1.0],
    'neuralnetwork__use_nesterov': [True, False],
    'neuralnetwork__weight_decay': [1e-3, 1e-2],
    'neuralnetwork__input_dimension': [10],
    'neuralnetwork__output_dimension': [3],
    'neuralnetwork__dropout_input_rate': [0.0, 0.1],
    'neuralnetwork__dropout_hidden_rate': [
        (0.1, 0.1),
        (0.2, 0.2),
        (0.3, 0.4)
    ],
    'neuralnetwork__patience': [9],
    'neuralnetwork__epochs': [400],
    'neuralnetwork__batch_size': [16, 32]
}

In [9]:
from sklearn.metrics import make_scorer

mee_scorer = make_scorer(mean_euclidean_error, greater_is_better=False)
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)
multidim_r2_scorer = make_scorer(multidim_r2, greater_is_better=True)

scoring = {'MEE': mee_scorer, 'R2': multidim_r2_scorer, 'RMSE': rmse_scorer}

In [10]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    refit='MEE',
    n_jobs=N_JOBS,
    verbose=4
)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 3888 candidates, totalling 19440 fits


2024-01-19 16:32:23.316547: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-19 16:32:23.317021: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-19 16:32:23.317035: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-19 16:32:23.317819: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-19 16:32:23.319045: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-01-19 16:32:23.437768: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-19 16:32:23.437858: I metal_plugin/src/device/metal_device.cc:296] system

KeyboardInterrupt: 

�       | 9.00/40.0 [00:00<00:00, 51.3batch/s, loss=31.4]
 75%|███████▌  | 30.0/40.0 [00:00<00:00, 79.5batch/s, loss=32.4]
 68%|██████▊   | 27.0/40.0 [00:00<00:00, 66.4batch/s, loss=11.2]
 50%|█████     | 20.0/40.0 [00:00<00:00, 63.5batch/s, loss=33.6]

 75%|███████▌  | 30.0/40.0 [00:00<00:00, 68.6batch/s, loss=11][A
 50%|█████     | 20.0/40.0 [00:00<00:00, 90.6batch/s, loss=11.4]
 90%|█████████ | 36.0/40.0 [00:00<00:00, 73.9batch/s, loss=11.1]
 45%|████▌     | 18.0/40.0 [00:00<00:00, 60.4batch/s, loss=31.1]
 98%|█████████▊| 39.0/40.0 [00:00<00:00, 74.7batch/s, loss=32.3]
 72%|███████▎  | 29.0/40.0 [00:00<00:00, 69.9batch/s, loss=33.8]
 15%|█▌        | 60/400 [00:42<04:00,  1.41epoch/s, loss=32.1]
  0%|          | 0.00/40.0 [00:00<?, ?batch/s, loss=32.1]       
 16%|█▌        | 64/400 [00:43<03:59,  1.40epoch/s, loss=10.8]  
  0%|          | 0.00/40.0 [00:00<?, ?batch/s, loss=10.8]     

 75%|███████▌  | 30.0/40.0 [00:00<00:00, 80.6batch/s, loss=11.4]


 62%|██████▎   | 25.0/40.0 [00:0

In [ ]:
best_params = grid_search.best_params_
best_index = grid_search.best_index_

mee = grid_search.best_score_
r2 = grid_search.cv_results_['mean_test_R2'][best_index]
rmse = grid_search.cv_results_['mean_test_RMSE'][best_index]

print('Best parameters:', best_params)
print('Best MEE score:', abs(mee))
print('R2:', abs(r2))
print('RMSE:', abs(rmse))

In [ ]:
final_model = grid_search.best_estimator_
final_model

# Learning Curve

In [ ]:
nn = final_model.named_steps['neuralnetwork']

history_df = pd.DataFrame(nn.history())
history_df.to_csv('history_loss_MEE.csv', index=False)

In [ ]:
history_df[['loss', 'val_loss']].plot(
    figsize=(8, 5),
    xlim=[0, 200],
    ylim=[0, 2],
    grid=True,
    xlabel='Epoch',
    style=['r--', 'b--'],
    title='Training and Validation Loss'
)
plt.show()

In [ ]:
history_df[['mean_euclidean_error', 'val_mean_euclidean_error']].plot(
    figsize=(8, 5),
    xlim=[0, 200],
    ylim=[0, 1],
    grid=True,
    xlabel='Epoch',
    style=['r--.', 'b--'],
    title='Training and Validation MEE'
)
plt.show()

# Test of the model

In [ ]:
y_pred = final_model.predict(X_test)

mee = mean_euclidean_error(y_test, y_pred)
r2 = multidim_r2(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)

print('MEE:', mee)
print('R2:', r2)
print('RMSE:', rmse)

# Save model

In [ ]:
model_path = os.path.join(MODEL_FOLDER, 'NN_model.joblib')
dump(final_model, model_path, compress=3)